In [1]:
import tweepy
from pprint import pprint
import jsonpickle
from tweepy import Stream
from tweepy.streaming import StreamListener
import time
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
auth = tweepy.OAuthHandler('iEk2lB2S9h6qdi9bG2F3OOfVX','LHOn4yNXL5ggl193syNeYaMs2jHGlw7zwDVLFc2pjVWW9XD5LO')
auth.set_access_token('1450808706-r0tIB7jGbOeHbz5EiXkUed2HdnTyNqZFGSZINTL','UCkmKWNSK6EYvk5MTT75GJERl7MPwGOMEXAsjqFnhmlBw')

api = tweepy.API(auth)
if (not api):
    print("Authentication failed :(")
else:
    print("Authentication successfull!!! :D")

Authentication successfull!!! :D


In [3]:
df = pd.read_csv(
    'corpus.txt', 
    delimiter='\t', 
    names=['event_mention_id', 'tweet_id', 'event_instance_id', 'event_mention_trigger', 'timestamp', 'text']
)

In [4]:
df1 = {
    'tweet_id':list(),
    'text':list()
}
tweets = []
dataset = []
tweet_ids = df['tweet_id'].values.tolist()

In [ ]:
for i in tqdm(range(0, len(tweet_ids), 100)):
    statuses = api.statuses_lookup(tweet_ids[i: i + 100])
    dataset.append(statuses)
    
    for idx, status in enumerate(statuses):
        tweet = df.iloc[[idx]]
        df1['tweet_id'].append(status._json['id'])
        df1['text'].append(status._json['text'])
    df_prime = pd.DataFrame(df1)
    df_prime.to_csv('fetched_tweets.csv')
    time.sleep((15 * 60) + 2)

In [5]:
import tokenizer as tk
import re

In [6]:
df1 = pd.DataFrame(df1)
def cleanTweet(tweet):
    s = " ".join(tk.tokenize(tweet[:-1]))
    x = re.sub(r'[^\x00-\xf3]', '', s)
    return x
df1['text'] = df1['text'].apply(cleanTweet)

In [7]:
df_main = {
    'event_mention_id': list(), 
    'tweet_id': list(), 
    'event_instance_id': list(), 
    'event_mention_trigger': list(), 
    'timestamp': list(), 
    'text':list()
}

In [8]:
for tweet_id in df1['tweet_id'].values:
    tweet = df[tweet_id == df['tweet_id']]
    for key in df_main.keys():
        if key != 'text':
            df_main[key].append(tweet[key].values[0])
    df_main['text'].append(df1[df1['tweet_id'] == tweet_id]['text'].values[0])

In [9]:
df_main = pd.DataFrame(df_main)

In [10]:
def transform(line):
    line = line.lower()
    line = line.replace("'''", "'")
    line = line.replace("isn't", "is not")
    line = line.replace("wasn't", "was not")
    line = line.replace("didn't", "did not")
    line = line.replace("don't", "do not")
    line = line.replace("dont", "do not")

    line = line.replace("doesn't", "does not")
    line = line.replace("doesnt", "does not")

    line = line.replace("won't", "will not")
    line = line.replace("wont", "will not")

    line = line.replace("wouldn't", "would not")
    line = line.replace("can't", "can not")
    line = line.replace("cannot", "can not")

    line = line.replace("hasn't", "has not")
    line = line.replace("*questioned", "questioned")
    line = line.replace("targets", "target")
    line = line.replace("=", "")
        
    return line


In [11]:
df_main['text'] = df_main['text'].apply(transform)

In [12]:
df_main.to_csv('FinalData.csv')

In [13]:
df_main = pd.read_csv(
    'generated_dataset.txt', 
    sep='\t', 
    names=[
        'event_mention_id', 
        'tweet_id', 
        'event_instance_id', 
        'event_mention_trigger', 
        'timestamp', 
        'text'
    ]
)

In [14]:
df_main

,event_mention_id,tweet_id,event_instance_id,event_mention_trigger,timestamp,text
0,0,784576649627103233,48,bragging,1475892624657,drools over a 12-year old and rapes a 13-year ...
1,1,784576649627103233,1514,rapes,1475892624657,drools over a 12-year old and rapes a 13-year ...
2,5,784579912812400641,17,is under attack,1475893402661,@thehill @wsj @washtimes @foxnews trump is und...
3,6,784579912812400641,1495,helping,1475893402661,@thehill @wsj @washtimes @foxnews trump is und...
4,8,784581821233242114,53,attack,1475893857664,republicans attack donald trump following emer...
...,...,...,...,...,...,...
1768,2807,791598375917486081,1097,does not endorse,1477566734662,boingboing : the yale record does not endorse ...
1769,2810,791626364486946816,1116,presses,1477573407657,clinton supporter presses trump surrogate on s...
1770,2811,791628209997680641,1116,presses,1477573847661,clinton supporter presses trump surrogate on s...
1771,2812,791638100149669888,1097,does not endorse,1477576205657,"quite funny semantics here . "" the yale record..."


# Word2Vec

In [15]:
from gensim.models import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile

In [16]:
class Tweet_Reader:
    def __init__(self, df):
        self.dataframe = df

    def __iter__(self):
        for idx in self.dataframe.index:
            temp = self.dataframe['text'][idx]
            temp = tk.tokenize(temp.lower())
            yield temp

In [17]:
path = get_tmpfile("word2vec.model")
tweets = Tweet_Reader (df_main) # a memory-friendly iterator

model = Word2Vec(tweets, size=100, window=5, min_count=1, workers=4, iter=30)
model
model.save("word2vec.model")

In [18]:
from torchtext.data import Field
from tqdm.notebook import tqdm
import torch
from torchtext.vocab import Vocab

## Construct Vocab

In [19]:
v = {}
for word, vocab_obj in model.wv.vocab.items():
    v[word] = vocab_obj.count
v['<unk>'] = 0
v['<pad>'] = 0

In [20]:
vc = Vocab(v, specials=['<pad>'])

In [21]:
word2vec_vectors = []
for token, idx in tqdm(vc.stoi.items()):
    if token in model.wv.vocab.keys():
        word2vec_vectors.append(torch.FloatTensor(model[token]))
    else:
        word2vec_vectors.append(torch.zeros(100))

/home/rizwan/anaconda3/envs/acads/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
/home/rizwan/anaconda3/envs/acads/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  after removing the cwd from sys.path.


In [22]:
vc.set_vectors(vc.stoi, word2vec_vectors, 100)

In [30]:
vc.stoi["'s"]

KeyError: "'s"

In [24]:
pre_trained_emb = torch.FloatTensor(vc.vectors)

In [25]:
embedding = torch.nn.Embedding.from_pretrained(pre_trained_emb)

In [26]:
embedding(torch.Tensor([vc.stoi['<pad>']]).long())

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]])

In [27]:
model.wv['httpurl']

array([ 0.3821557 , -0.4972439 , -0.5901334 , -0.08836859, -0.27361524,
       -0.48360226,  0.24551167, -0.6968162 ,  0.33877534, -0.5088345 ,
        0.09811841, -0.07253633,  0.4954285 , -0.04667998,  0.02213344,
        0.15480609,  0.00363866, -0.5061666 , -0.6533979 , -0.9323445 ,
       -0.01032262, -0.34816974, -0.36009434,  0.14922507,  0.12657423,
        0.56726134, -0.9851585 , -0.12662159, -0.77473825, -0.2708073 ,
        0.00707218, -0.71529   , -0.2839096 , -0.30382675, -0.39199585,
       -0.5219222 ,  0.34463128, -0.13196824,  0.08685283,  0.13087487,
       -0.590358  ,  0.18899724,  0.31627762, -0.01037497,  0.77803135,
        0.29710972, -0.07571774, -0.3354809 , -0.5275393 , -0.43512473,
        0.18143053, -0.1266352 ,  0.29629534, -0.50368154, -0.12777792,
       -0.5524746 , -1.25475   ,  0.13420637,  0.34761652,  0.39997756,
       -0.16451469, -0.14968158, -0.2357309 , -0.00163437,  0.1808891 ,
       -0.3058456 , -0.01153537, -0.14755237, -0.5672787 , -0.91

In [28]:
import pickle

In [29]:
with open("vocab.pkl", 'wb') as f:
    pickle.dump(vc, f)